# Preprocessing

In [2]:
# Setup notebook
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
import datetime
import math

In [1]:
# Ignore Future Warning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [14]:
# Setup
train = pd.read_csv('train.csv', parse_dates=["date"])
test = pd.read_csv('test.csv', parse_dates=["date"])

In [15]:
print(train.shape, test.shape)
# train.head()
test.head()

(3000888, 6) (28512, 5)


,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [51]:
# # Grouping by "date" and "family"
# train_df = train.groupby(by=["id", "family"]).sum()
# train_df.head()

# print(train_df .shape)
# train_df[-5:]

In [39]:
# # Grouping by "date" and "family"
# test_df = test.groupby(by=["id", "family"]).sum()
# test_df.head()

# print(test_df .shape)
# test_df.head()

In [11]:
# Error Function (RMSLE)

def error(y_p, y_t):
    # y_p: predictted (fitted)
    # y_t: test (actual)
    pred_log = np.array([math.log(i+1) for i in np.nditer(y_p)])
    act_log = np.array([math.log(i+1) for i in np.nditer(y_t)])
    dum_error = sum((pred_log - act_log)**2)/len(pred_log)
    linear_error = np.power(dum_error, 1/2)
    return round(linear_error, 4)

# Blue print
pre : separate train data set into "ourtrain" and "ourtest" (**sorted**, 70:30)
1. Make subsets for each family
2. Apply ML models to all subsets
3. Combine the subsets
4. Compute error function
5. Choose the ML model which gives the min error
6. Apply to test data set

In [40]:
# Separate train data set into "ourtrain" and "ourtest" (sorted, 70:30)

from sklearn.model_selection import train_test_split

ourtrain, ourtest = train_test_split(train, test_size=0.3, random_state=None, shuffle=False)

In [41]:
# Check ourtrain
print(ourtrain.shape)
ourtrain.head()

(2100621, 6)


,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [44]:
# Check ourtest
print(ourtest.shape)
print(len(ourtrain), "+", len(ourtest), "=", len(train.index))
ourtest.head()

(900267, 6)
2100621 + 900267 = 3000888


,id,date,store_nbr,family,sales,onpromotion
2100621,2100621,2016-03-27,49,CELEBRATION,32.000,0
2100622,2100622,2016-03-27,49,CLEANING,1977.000,26
2100623,2100623,2016-03-27,49,DAIRY,3036.000,0
2100624,2100624,2016-03-27,49,DELI,519.661,0
2100625,2100625,2016-03-27,49,EGGS,583.000,0


# 1. Make subsets of *ourtrain* for each family

In [139]:
# Check unique values of "family"
family_list = train["family"].unique()
print(family_list, "\nNumber of unique values:", len(family_list))

['AUTOMOTIVE' 'BABY CARE' 'BEAUTY' 'BEVERAGES' 'BOOKS' 'BREAD/BAKERY'
 'CELEBRATION' 'CLEANING' 'DAIRY' 'DELI' 'EGGS' 'FROZEN FOODS' 'GROCERY I'
 'GROCERY II' 'HARDWARE' 'HOME AND KITCHEN I' 'HOME AND KITCHEN II'
 'HOME APPLIANCES' 'HOME CARE' 'LADIESWEAR' 'LAWN AND GARDEN' 'LINGERIE'
 'LIQUOR,WINE,BEER' 'MAGAZINES' 'MEATS' 'PERSONAL CARE' 'PET SUPPLIES'
 'PLAYERS AND ELECTRONICS' 'POULTRY' 'PREPARED FOODS' 'PRODUCE'
 'SCHOOL AND OFFICE SUPPLIES' 'SEAFOOD'] 
Number of unique values: 33


In [168]:
# Generates subsets for each family
def generate_subsets(family):
    subset = ourtrain.loc[train['family']==family, :]
    return subset

# Generate all subsets
subsets = {}
for family in family_list:
    subsets.update({family:generate_subsets(family)})

# Check
subsets['BABY CARE'].head()

,id,date,store_nbr,family,sales,onpromotion
1,1,2013-01-01,1,BABY CARE,0.0,0
34,34,2013-01-01,10,BABY CARE,0.0,0
67,67,2013-01-01,11,BABY CARE,0.0,0
100,100,2013-01-01,12,BABY CARE,0.0,0
133,133,2013-01-01,13,BABY CARE,0.0,0


In [170]:
# Extracts features
def ext_features(df):
    return 

# Extracts targets
def ext_targets(df):
    return

# 2. Apply ML models to all subsets

In [ ]:
# Feature

# 3. Combine the subsets

# 4. Compute error function

# 5. Choose the ML model which gives the min error

# 6. Apply to test data set